## TrOCR Fine Tune


##### Install Necessary Libraries

In [1]:
# %pip install -q torch torchvision torchaudio
# %pip install -q datasets jiwer

##### Import Necessary Libraries

In [2]:
import os, sys, itertools
os.environ['TOKENIZERS_PARALLELISM']='false'
# export CUDA_LAUNCH_BLOCKING=1
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split

import cv2
from PIL import Image

import torch
from torch.utils.data import Dataset

import datasets
from datasets import load_dataset

import transformers
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import VisionEncoderDecoderModel, TrOCRProcessor, default_data_collator
from datasets import load_metric


from preprocess_image import PreprocessImage
from augment_image import AugmentImage

##### Ingest, Preprocess, & Split Dataset (into Training & Testing Datasets)

In [3]:
df_train = pd.read_csv('train/DataTrain.csv', delimiter=';')
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_train = df_train.drop([126, 457, 600]) # delete the wrong labeled data
df_train.head()

,Vehicleregistrationplate,NameofFile
0,A7814,DataTrain1.png
1,B1074QO,DataTrain2.png
2,B1031QO,DataTrain3.png
3,B187EDA,DataTrain4.png
4,B1089VD,DataTrain5.png


##### Show First Samples in DataFrame

In [4]:
df_train.head(12)

,Vehicleregistrationplate,NameofFile
0,A7814,DataTrain1.png
1,B1074QO,DataTrain2.png
2,B1031QO,DataTrain3.png
3,B187EDA,DataTrain4.png
4,B1089VD,DataTrain5.png
5,B1972RBP,DataTrain6.png
6,AB2400WU,DataTrain7.png
7,AB6268YQ,DataTrain8.png
8,A8014VA,DataTrain9.png
9,B1554EJA,DataTrain10.png


##### Prepare the image and label dataset

###### Prepare Label & Images

In [5]:
label_array = df_train['Vehicleregistrationplate'].to_numpy()
path_train = df_train['NameofFile'].to_numpy()

###### Train val split

In [6]:
train_path_array, test_path_array, train_label_array, test_label_array = train_test_split(path_train, label_array, test_size=0.2, random_state=42)

###### Preprocess Test Images

In [7]:
test_image = PreprocessImage(img_paths=test_path_array, output_pixel=384, padding_ratio=0.9, path_prefix='train', include_original=False)
test_img_array = test_image.img_array.copy()

Initial Preprocess: Add Padding:   0%|          | 0/160 [00:00<?, ?it/s]

Initial Preprocess: Add Padding: 100%|██████████| 160/160 [00:00<00:00, 685.78it/s]


###### Preprocess & Augment Train Images

In [8]:
train_image = PreprocessImage(img_paths=train_path_array, output_pixel=384, padding_ratio=0.9, path_prefix='train', include_original=True)
# train_image_array = train_image.gray_image(masked_img=np.array([]))
train_img_array = train_image.img_array.copy()
print(train_img_array.shape)

Initial Preprocess: Add Padding: 100%|██████████| 637/637 [00:00<00:00, 709.24it/s]


(637, 384, 384, 3)


In [9]:
# train_label_array = np.concatenate((train_label_array, train_label_array), axis=0)

In [10]:
train_label_array.shape

(637,)

##### Create Dataset Class

In [11]:
class BDC_Dataset(Dataset):
    
    def __init__(self, img_array, label_array, processor, max_target_length=12):
        self.img_array = img_array
        self.label_array = label_array
        self.processor = processor
        self.max_target_length = max_target_length
    
    def __len__(self):
        return len(self.img_array)

    def __getitem__(self, idx):
        # get iimage + label
        image = self.img_array[idx]
        label = self.label_array[idx]
        # prepare image (i.e. resize + normalize)
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        pixel_values = self.processor(image, return_tensors="pt").pixel_values.to('mps:0')
        # add labels (input_ids) by encoding the label
        labels = self.processor.tokenizer(label, padding="max_length", max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id 
                  else -100 for label in labels]
        
        encoding = {"pixel_values" : pixel_values.squeeze(), "labels" : torch.tensor(labels).to('mps:0')}
        return encoding

##### Basic Values/Constants

In [12]:
MODEL_CKPT = "microsoft/trocr-base-printed"
MODEL_NAME =  MODEL_CKPT.split("/")[-1] + "_bdc_license_plates_ocrV2"
NUM_OF_EPOCHS = 2
BATCH_SIZE = 4

##### Instantiate Processor, Create Training, & Testing Dataset Instances

In [13]:
processor = TrOCRProcessor.from_pretrained(MODEL_CKPT)


test_ds = BDC_Dataset(test_img_array, test_label_array, processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


##### Print Length of Training & Testing Datasets

In [14]:
# print(f"The training dataset has {len(train_ds)} samples in it.")
print(f"The testing dataset has {len(test_ds)} samples in it.")

The testing dataset has 160 samples in it.


##### Example of Input Data Shapes

In [15]:
# encoding = train_ds[0]

# for k,v in encoding.items():
#     print(k, " : ", v.shape)

##### Show Example

In [16]:
# image = Image.open(train_ds.root_dir + train_dataset['file_name'][0]).convert("RGB")

# image

##### Show Label for Above Example

In [17]:
# labels = encoding['labels']
# labels[labels == -100] = processor.tokenizer.pad_token_id
# label_str = processor.decode(labels, skip_special_tokens=True)
# print(label_str)

#### Instantiate Model

In [18]:
from torch.utils.data import DataLoader


eval_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [19]:
model = VisionEncoderDecoderModel.from_pretrained(MODEL_CKPT)
device = 'mps:0'
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fea

##### Model Configuration Modifications

In [20]:
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

model.config.vocab_size = model.config.decoder.vocab_size

model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 12
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

##### Define Metrics Evaluation

In [24]:
from metrics import character_accuracy_np as char_acc 

cer_metric = load_metric("cer")

def compute_metrics(pred_ids, label_ids):

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)
    
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    # tmp_hat_test = [np.asarray(list(x)) for x in pred_str]
    # pred_str = np.asarray(tmp_hat_test)
    # tmp_label_test = [np.asarray(list(x)) for x in label_str]
    # label_str = np.asarray(tmp_label_test)
    # acc = char_acc(label_str, pred_str)

    return cer

/var/folders/vt/q06w72ls4kz7lg_wsqqw2f0h0000gp/T/ipykernel_96591/3840881219.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")


In [22]:
from tqdm import tqdm
def manual_eval(model, processor, images):
    output = []
    px_vals = []
    for image in tqdm(images):
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
        px_vals.append(pixel_values)
    for pixel_values in tqdm(px_vals):
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        output.append(generated_text)
    return np.asarray(output)

In [23]:
from transformers import AdamW
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR


optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=6.5e-6, no_deprecation_warning=True)
max_epoch = 5
scheduler = StepLR(optimizer, step_size=2, gamma=0.85)


prev_loss = 2e5
for epoch in range(max_epoch):  # loop over the dataset multiple times
    num_aug = 7-(epoch*2)
    if num_aug < 1:
        num_aug = 1
    # Generate augmented image data for each epoch
    if epoch >= 0:
        augmenter = AugmentImage(image_array=train_img_array, label_array=train_label_array, num_augmentations=num_aug)
        train_image_array, train_lbl_array = augmenter.transform()
    else:
        train_image_array = train_img_array
        train_lbl_array = train_label_array
    train_ds = BDC_Dataset(train_image_array, train_lbl_array, processor=processor)
    train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    # train
    model.train()
    train_loss = 0.0
    early_stop = False
    for idx, batch in enumerate(tqdm(train_dataloader, desc=f'Epoch: {epoch+1}/{max_epoch}')):
        # get the inputs
        for k,v in batch.items():
            batch[k] = v.to(device)

        # forward + backward + optimize
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()
        
        if idx % (1 + (len(train_dataloader)//10)) == 0 and idx != 0:
            prev_loss = train_loss/(idx+1)
            print(f"Loss after batch {(idx+1)}:", train_loss/(idx+1))
            for i in range(epoch+1):
                scheduler.step()
        if prev_loss*1.2 < train_loss/(idx+1):
            early_stop = True
            break
    if early_stop:
        print(f"Loss after epoch {epoch}:", train_loss/(idx+1))
        prev_loss = train_loss/(idx+1)
    else:
        print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))
        prev_loss = train_loss/(idx+1)
    for i in range((max_epoch+epoch-2)*5):
        scheduler.step()
    # evaluate
    model.eval()
    valid_cer = 0.0
    valid_acc = 0.0
    with torch.no_grad():
        yhat_test = manual_eval(model=model, processor=processor, images = test_img_array)
        tmp_hat_test = [np.asarray(list(x)) for x in yhat_test]
        yhat_test = np.asarray(tmp_hat_test)
        tmp_label_test = [np.asarray(list(x)) for x in test_label_array]
        ylabel_test = np.asarray(tmp_label_test)
        valid_acc = char_acc(ylabel_test, yhat_test)

        for batch in tqdm(eval_dataloader):
            # run batch generation
            outputs = model.generate(batch["pixel_values"].to(device))
            # compute metrics
            cer = compute_metrics(pred_ids=outputs, label_ids=batch["labels"])
            valid_cer += cer 
        

    print("Validation CER:", valid_cer / len(eval_dataloader))
    print("Validation Char_acc:", valid_acc)


/Users/mdaniyalk/miniforge3/lib/python3.10/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)
Augment Images:   0%|          | 0/637 [00:00<?, ?it/s]

Epoch: 1/5:  10%|█         | 113/1115 [05:52<56:11,  3.36s/it] 

Loss after batch 113: 1.032955126556675


Epoch: 1/5:  20%|██        | 225/1115 [11:41<48:16,  3.25s/it]  

Loss after batch 225: 0.8649246018462711


...

Epoch: 1/5:  40%|████      | 449/1115 [24:32<37:31,  3.38s/it]

Loss after batch 449: 0.6765433798040075


Epoch: 1/5:  50%|█████     | 561/1115 [31:04<30:06,  3.26s/it]

Loss after batch 561: 0.5831750251034046


Epoch: 1/5:  52%|█████▏    | 585/1115 [32:25<32:09,  3.64s/it]

In [24]:
outputs.loss

AttributeError: 'Tensor' object has no attribute 'loss'

In [24]:
valid_acc

0.9381944444444444

In [25]:
model.save_pretrained(f'{MODEL_NAME}d')

In [ ]:
del model

##### Model Evaluate

In [9]:
from tqdm import tqdm
def manual_eval(model, processor, images):
    output = []
    px_vals = []
    for image in tqdm(images):
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        pixel_values = processor(image, return_tensors="pt").pixel_values.to('cpu')
        px_vals.append(pixel_values)
    for pixel_values in tqdm(px_vals):
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        output.append(generated_text)
    return np.asarray(output)

In [20]:
bdc_model = VisionEncoderDecoderModel.from_pretrained('trocr-base-printed_bdc_license_plates_ocrV2f').to('mps')


In [11]:
from metrics import character_accuracy_np

# yhat_train = manual_eval(model=bdc_model, processor=processor, images = train_image_array)
# tmp_hat = [np.asarray(list(x)) for x in yhat_train]
# yhat_train = np.asarray(tmp_hat)
# print(f"Train dataset char_acc = {character_accuracy_np(train_label_array, yhat_train)}")

yhat_test = manual_eval(model=model, processor=processor, images = test_img_array)
tmp_hat_test = [np.asarray(list(x)) for x in yhat_test]
yhat_test = np.asarray(tmp_hat_test)
tmp_label_test = [np.asarray(list(x)) for x in test_label_array]
ylabel_test = np.asarray(tmp_label_test)
print(f"test dataset char_acc = {character_accuracy_np(ylabel_test, yhat_test)}")

NameError: name 'test_img_array' is not defined

In [27]:
df_gt = pd.read_csv('/Users/mdaniyalk/Downloads/marsel-ground_truth_manual.csv')
gt = df_gt['Ground Truth'].to_numpy()
gt_test_path_array = df_gt['Name of File'].to_numpy()

gt_test_image = PreprocessImage(img_paths=gt_test_path_array, output_pixel=384, padding_ratio=0.99, path_prefix='test', include_original=False)
gt_test_img_array = gt_test_image.img_array.copy()
# gt_test_img_array = gt_test_image.gray_image(masked_img=np.array([]))
yhat_test = manual_eval(model=bdc_model, processor=processor, images = gt_test_img_array)

tmp_hat_test = [np.asarray(list(x)) for x in yhat_test]
yhat_test = np.asarray(tmp_hat_test)
tmp_label_test = [np.asarray(list(x)) for x in gt]
ylabel_test = np.asarray(tmp_label_test)
print(f"test dataset compared to GroundTruth char_acc = {char_acc(ylabel_test, yhat_test)}")


  0%|          | 0/100 [00:00<?, ?it/s]/Users/mdaniyalk/miniforge3/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (12) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 100/100 [02:37<00:00,  1.57s/it]

test dataset compared to GroundTruth char_acc = 0.9655555555555554



/var/folders/vt/q06w72ls4kz7lg_wsqqw2f0h0000gp/T/ipykernel_96591/2541768004.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  yhat_test = np.asarray(tmp_hat_test)
/var/folders/vt/q06w72ls4kz7lg_wsqqw2f0h0000gp/T/ipykernel_96591/2541768004.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ylabel_test = np.asarray(tmp_label_test)


In [30]:
df_gt = pd.read_csv('/Users/mdaniyalk/Downloads/marsel-ground_truth_manual.csv')
gt = df_gt['Ground Truth'].to_numpy()

tmp_label_test = [np.asarray(list(x)) for x in gt]
ylabel_test = np.asarray(tmp_label_test)
print(f"test dataset compared to GroundTruth char_acc = {char_acc(ylabel_test, yhat_test)}")


test dataset compared to GroundTruth char_acc = 0.9655555555555554


/var/folders/vt/q06w72ls4kz7lg_wsqqw2f0h0000gp/T/ipykernel_96591/3795801008.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ylabel_test = np.asarray(tmp_label_test)


In [ ]:
test = []
for gt_y in yhat_test:
    tmp = []
    for aa in gt_y:
        if aa != '[PAD]':
            tmp.append(aa)
            
    str_value = ''.join(tmp)
    test.append(str_value)
test = np.asarray(test)
test


array(['AD7034BE', 'A9388EX', 'B16TB', 'B1661TKZ', 'AD3772ABE', 'B1373HV',
       'B1064TFR', 'B1395TJW', 'B1270RFD', 'B1736BYH', 'B1627BIE',
       'B1678WZM', 'AD9313SS', 'B1036UL', 'B1801TZS', 'B1474TJS',
       'B1939PU', 'B1260TZT', 'B1376TJB', 'B17QO', 'AB6632ULZ',
       'B1236PZM', 'AB8644PK', 'B1KT', 'AA7084QO', 'B1131EKG', 'B1163TJN',
       'B1036UL', 'AB8820CB', 'A9192ZM', 'B0120KXT', 'B1643TRO',
       'B1390TJQ', 'AB2681VD', 'B1860EFT', 'A8749FS', 'B1566FON',
       'B1063SJQ', 'B1254TFX', 'B1038JY', 'AB4923QH', 'B1509UN',
       'B1937TLP', 'B1202UL', 'B1026TMZ', 'B1724PYW', 'B1102SIV',
       'AD7034OF', 'AB4352CX', 'B1895EJB', 'B1786UJT', 'B1549RFS',
       'B1869EOF', 'B1713VY', 'B1063SPW', 'B1661TKZ', 'A8014VA',
       'B1873YU', 'B1820UL', 'B1422BKI', 'AB5278XA', 'AD418U', 'B1157Y',
       'B1233RFD', 'B1031NI', 'AD99JR', 'B1683SEY', 'AA7014OF',
       'B1241SSW', 'B1632TJJ', 'B1907ELR', 'B1815TJQ', 'B1734UJN',
       'B1743EYF', 'B1075QO', 'AB2933TM', 'B1523TJT', '

In [22]:
subm = pd.read_csv('submission_sample.csv')
subm = subm.drop(['Unnamed: 0'], axis=1)

subm.head()

,Name of File,Vehicleregistrationplatebymodel
0,DataTest1.png,NaN
1,DataTest2.png,NaN
2,DataTest3.png,NaN
3,DataTest4.png,NaN
4,DataTest5.png,NaN


In [23]:
subm['Vehicleregistrationplatebymodel'] = test
subm.head()

,Name of File,Vehicleregistrationplatebymodel
0,DataTest1.png,AD7034OF
1,DataTest2.png,A9388EX
2,DataTest3.png,B16TB
3,DataTest4.png,B1661TKZ
4,DataTest5.png,AD3772ABE


In [25]:
subm.to_csv('submission_modelV1.csv', index=True)

### Notes & Other Takeaways From This Project
****
- The results were pretty good. I was pondering whether to train for 2 or 3 epochs. Ultimately, I trained this model for 2 epochs. If this were a work project (where multiprocessing and other options are available), I would have trained for 3, if not 4, epochs.
****

### Citations

##### For Transformer Checkpoint
- @misc{li2021trocr,
      title={TrOCR: Transformer-based Optical Character Recognition with Pre-trained Models}, 
      author={Minghao Li and Tengchao Lv and Lei Cui and Yijuan Lu and Dinei Florencio and Cha Zhang and Zhoujun Li and Furu Wei},
      year={2021},
      eprint={2109.10282},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}

##### For CER Metric
- @inproceedings{morris2004,
author = {Morris, Andrew and Maier, Viktoria and Green, Phil},
year = {2004},
month = {01},
pages = {},
title = {From WER and RIL to MER and WIL: improved evaluation measures for connected speech recognition.}
}